We want to make the transition from using ML algorithms like black boxes, to more deeply understanding what's happening under the hood. This will allow us to:
- Home in on the appropriate model, training algorithm, and hyperparameters. 
- Debug issues and perform error analysis more efficiently. 
- Build and train neural networks. 

We'll start by looking at Linear Regression, and two different ways to train it:
- Using a "closed-form" equation. 
- Using "Gradient Descent". 